## Overview

This example demonstrates the usage of a GramART module.
This module is tested on a modified symbolic Iris dataset as a proof of concept, but it is capable of working on arbitrary symbolic datasets.

## Setup

First, we load some dependencies:

In [1]:
# Import the OAR project module
using OAR

Next, we can load the Iris dataset in a modified symbolic form:

In [2]:
# All-in-one function
fs, bnf = OAR.symbolic_iris()

(OAR.VectoredDataSplit{GSymbol{String}, Int64}: dim=4, n_train=105, n_test=45:
train_x: (105,) Vector{Vector{GSymbol{String}}}
test_x: (45,) Vector{Vector{GSymbol{String}}}
train_y: (105,) Vector{Int64}
test_y: (45,) Vector{Int64}
, OAR.CFG{String}(N:4, S:4, P:4, T:40))

We can finally initialize the GramART module using the grammar that we have describing the symbolic Iris dataset:

In [3]:
# Initialize the GramART module
gramart = OAR.GramART(bnf)

GramART(ProtoNode[], OAR.CFG{String}(N:4, S:4, P:4, T:40), OAR.opts_GramART
  rho: Float64 0.7
  rho_lb: Float64 0.55
  rho_ub: Float64 0.75
  alpha: Float64 0.001
  beta: Float64 1.0
  max_epoch: Int64 1
  terminated: Bool false
, Int64[], Dict{String, Any}("n_categories" => 0, "n_clusters" => 0, "n_instance" => Int64[]))

## Training

Now that we have a GramART module, we should process the training dataset:

In [4]:
# Cluster the statements
for statement in fs.train_x
    OAR.train!(gramart, statement)
end

In fact, we can also do a simple supervised version of the training if labels are available.
Let's do that with another module:

In [5]:
# Initialize the GramART module
gramart_supervised = OAR.GramART(bnf)
# Set the vigilance low for generalization
gramart_supervised.opts.rho = 0.05
# Train in supervised mode
for ix in eachindex(fs.train_x)
    sample = fs.train_x[ix]
    label = fs.train_y[ix]
    OAR.train!(gramart_supervised, sample, y=label)
end

## Analysis

Now let's see what's inside the first module:

In [6]:
# Inspect the module
@info "Number of categories: $(length(gramart.protonodes))"

[ Info: Number of categories: 50


We can also see how the supervised training went by classifying the test data and computing the performance:

In [7]:
# Classification
y_hat = zeros(Int, length(fs.test_y))
for ix in eachindex(fs.test_x)
    sample = fs.test_x[ix]
    y_hat[ix] = OAR.classify(gramart_supervised, sample, get_bmu=true)
end

# Calculate performance
perf = OAR.AdaptiveResonance.performance(y_hat, fs.test_y)
@info "Supervised testing performance: $(perf)"

[ Info: Supervised testing performance: 0.9555555555555556


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*